In [1]:
!pip install ipykernel 
!conda init bash
!module load wmlce/1.6.2

Defaulting to user installation because normal site-packages is not writeable
no change     /opt/miniconda3/condabin/conda
no change     /opt/miniconda3/bin/conda
no change     /opt/miniconda3/bin/conda-env
no change     /opt/miniconda3/bin/activate
no change     /opt/miniconda3/bin/deactivate
no change     /opt/miniconda3/etc/profile.d/conda.sh
no change     /opt/miniconda3/etc/fish/conf.d/conda.fish
no change     /opt/miniconda3/shell/condabin/Conda.psm1
no change     /opt/miniconda3/shell/condabin/conda-hook.ps1
no change     /opt/miniconda3/lib/python3.9/site-packages/xontrib/conda.xsh
no change     /opt/miniconda3/etc/profile.d/conda.csh
no change     /home/dbk3/.bashrc
No action taken.


In [2]:

# -*- coding: utf-8 -*-
"""ASL_Aspire_Mediapipe.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1wUto4Bvi_IYRPQ4Ph0jgNtkjKJaPdzr_

Usage example of MediaPipe Hands Solution API in Python (see also http://solutions.mediapipe.dev/hands).
"""

#**


#**
!pip install keras

!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:

#import extract_features
# Import the much needed stuff for training
import datetime
import time
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import os
import csv
import cv2 as cv
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.utils import to_categorical
import tensorflow.keras
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)



In [4]:
df_valtest = pd.read_csv("valdatafile.csv", header=0,error_bad_lines=False)

df_valtest = df_valtest.drop([1, 267, 417, 1124, 1362, 926, 1156, 1916])

df_valtest.to_csv('edited_valdatafile.csv', index=False)

In [5]:


"""#Reading and Loading in the Extracted Data

##Train Data Loading:
"""

# Read CSV file for Training the model using Pandas
df_train = pd.read_csv("traindatafile.csv", header=0, error_bad_lines=False)


# First we must sort the values of the dataset according to the Alphabets
df_train = df_train.sort_values(by=["class_type"]) #Every letter has a different class type. Sorts numerically.


"""##Validation Data Loading:"""

##**Run from this point onwards

# Read CSV file for Validation or Testing the Model using Pandas
df_val = pd.read_csv("edited_valdatafile.csv", header=0,error_bad_lines=False)

# First we must sort the values of the dataset according to the Alphabets
df_val = df_val.sort_values(by=["class_type"])



In [6]:
label_counts_train = df_train['class_type'].value_counts()
label_counts_val = df_val['class_type'].value_counts()
total_train_size = 0
total_val_size = 0
for x in label_counts_train:
    total_train_size += x
for y in label_counts_val: 
    total_val_size += y
    
print(label_counts_train)
print(label_counts_val)

print(total_train_size, total_val_size)

F    2956
K    2725
D    2722
V    2715
G    2707
S    2699
R    2690
U    2684
L    2664
Y    2634
W    2558
I    2541
H    2511
E    2458
O    2437
T    2413
B    2395
X    2393
A    2388
C    2366
Q    2266
M    2219
P    2211
N    1849
Name: class_type, dtype: int64
L    151
O    151
I    150
K    150
Y    150
S    150
F    150
G    149
R    147
C    145
H    139
T    124
P    122
D    120
Q    119
A    109
M     95
W     92
N     77
U     58
X     44
E     41
V     32
B     23
Name: class_type, dtype: int64
60201 2688


In [7]:


"""##Data Configuration for Model Input"""

# Put Categorical using Pandas
#Categoricals can only take on only a limited, and usually fixed, number of possible values (categories)
#All values of the Categorical are either in categories or np.nan
df_train["class_type"] = pd.Categorical(df_train["class_type"]) #Creates a 'categorical' with the values being the inputed class_Type from above
df_train["class_type"] = df_train.class_type.cat.codes #Accesses the class type values, assigns a code to it
#Later, this will correspond to the 0-26 A-Z letters

df_val["class_type"] = pd.Categorical(df_val["class_type"])
df_val["class_type"] = df_val.class_type.cat.codes

# Copy Label and Feature for training
y_train = df_train.pop("class_type") #Copies the df_val MINUS the class type indexes (i.e the landmarks)
x_train = df_train.copy()

y_val = df_val.pop("class_type") #Same
x_val = df_val.copy()

# Copied Features turn to Array by using NumPy
x_train = np.array(x_train)
x_val = np.array(x_val)


# Since the array shape is 1x10, we must turn it into 1x10x1 so we can feed it into the model
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_val = np.reshape(x_val, (x_val.shape[0], x_val.shape[1], 1))


# Number of classes according standard American Language Alphabets
num_classes = 24

# Using the Keras.Utils to put the label categorically 
y_train = to_categorical(y_train, num_classes)
y_val = to_categorical(y_val, num_classes)


In [8]:
# Check the number of available GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
numGpus = len(gpus)
print(numGpus)

4


In [9]:
"""Model and Training: on several GPUs"""


# One Dimensional Convolutional Neural Network model, Train will be feed to 1 Dimension Convolutional Neural Network

#TODO: Look into separable convolution
#TODO: Try different variations of ELU



strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv1D(filters=32, kernel_size=5, strides=1, padding="causal", activation="relu", input_shape=x_train.shape[1:3]),
        tf.keras.layers.Conv1D(filters=32, kernel_size=5, strides=1, padding="causal", activation="relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling1D(pool_size=2),  #added by Daniel
        tf.keras.layers.Conv1D(filters=64, kernel_size=5, strides=1, padding="causal", activation="relu"),
        tf.keras.layers.Conv1D(filters=64, kernel_size=5, strides=1, padding="causal", activation="relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling1D(pool_size=2),  #added by Daniel
        tf.keras.layers.Conv1D(filters=128, kernel_size=5, strides=1, padding="causal", activation="relu"),
        tf.keras.layers.Conv1D(filters=128, kernel_size=5, strides=1, padding="causal", activation="relu"),
        tf.keras.layers.BatchNormalization(),  #added by Daniel
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.Conv1D(filters=256, kernel_size=5, strides=1, padding="causal", activation="relu"),
        tf.keras.layers.Conv1D(filters=256, kernel_size=5, strides=1, padding="causal", activation="relu"),
        tf.keras.layers.BatchNormalization(),  ##added by Daniel
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.Dropout(rate=0.25),
        # Flatten the results to feed into a DNN
        tf.keras.layers.Flatten(),
        # 512 neuron hidden layer
        tf.keras.layers.Dense(512, activation='relu'), 
        tf.keras.layers.BatchNormalization(),  ##added by Daniel
        tf.keras.layers.Dense(num_classes, activation='softmax')])

    model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    

W0523 22:02:55.015190 140735892507248 deprecation.py:506] From /opt/miniconda3/envs/wmlce-v1.6.2/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
W0523 22:02:55.862542 140735892507248 backend.py:548] OMP_NUM_THREADS is no longer used by the default Keras config. To configure the number of threads, use tf.config.threading APIs.


In [10]:
# model = tf.keras.models.Sequential([
#     tf.keras.layers.Conv1D(filters=32, kernel_size=5, strides=1, padding="causal", activation="relu", input_shape=x_train.shape[1:3]),
#     tf.keras.layers.Conv1D(filters=32, kernel_size=5, strides=1, padding="causal", activation="relu"),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.MaxPooling1D(pool_size=2),  #added by Daniel
#     tf.keras.layers.Conv1D(filters=64, kernel_size=5, strides=1, padding="causal", activation="relu"),
#     tf.keras.layers.Conv1D(filters=64, kernel_size=5, strides=1, padding="causal", activation="relu"),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.MaxPooling1D(pool_size=2),  #added by Daniel
#     tf.keras.layers.Conv1D(filters=128, kernel_size=5, strides=1, padding="causal", activation="relu"),
#     tf.keras.layers.Conv1D(filters=128, kernel_size=5, strides=1, padding="causal", activation="relu"),
#     tf.keras.layers.BatchNormalization(),  #added by Daniel
#     tf.keras.layers.MaxPooling1D(pool_size=2),
#     tf.keras.layers.Conv1D(filters=256, kernel_size=5, strides=1, padding="causal", activation="relu"),
#     tf.keras.layers.Conv1D(filters=256, kernel_size=5, strides=1, padding="causal", activation="relu"),
#     tf.keras.layers.BatchNormalization(),  ##added by Daniel
#     tf.keras.layers.MaxPooling1D(pool_size=2),
#     tf.keras.layers.Dropout(rate=0.25),
#     # Flatten the results to feed into a DNN
#     tf.keras.layers.Flatten(),
#     # 512 neuron hidden layer
#     tf.keras.layers.Dense(512, activation='relu'), 
#     tf.keras.layers.BatchNormalization(),  ##added by Daniel
#     tf.keras.layers.Dense(num_classes, activation='softmax')])

# model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
#setting up the datasheet with all model's performance:

#NOTE: ONLY RUN THIS AT THE START OF TRAINING (AT EPOCH 0) OTHERWISE YOU REWRITE ALL PRIOR DATA
csv_file = 'Model_Performance.csv'

header = ["epoch", "training_loss", "training_acc", "val_loss", "val_acc"]

with open(csv_file, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(header)


In [ ]:
#Check for GPU availability
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Commented out IPython magic to ensure Python compatibility.
#Train the Model on the CPU for comparison
# startTime = time.time()
# with tf.device('/CPU:0'):
#   model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val),callbacks=[tensorboard_callback])
# endTime = time.time()
# print("Total Time Elapsed for CPU training:" )
# print(endTime - startTime)



#Train the Model on the GPU
#Steps for running on GPU: 
#1: Connect to Hosted Runtime from carrot on righthand side
#2: Edit->Notebook Settings -> GPU
from tensorflow.keras.callbacks import ModelCheckpoint
filepath="saved_models/model_epoch{epoch:02d}.hdf5"

checkpoint_callback = ModelCheckpoint(filepath, 
                                      monitor='val_accuracy', 
                                      verbose=1,
                                      save_best_only=False,
                                      save_freq='epoch',
                                      period=25)

class LossAndErrorPrintingCallback(tf.keras.callbacks.Callback):
    #runs at the end of each epoch and appends the values to the Model_Performance.csv
    def on_epoch_end(self, epoch, logs=None):
        
        row = []
        row.extend((epoch, logs["loss"], logs["acc"], logs["val_loss"], logs["val_acc"]))

        # Write the array to the CSV file
        with open(csv_file, 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(row)
        


# NOTE, tensorboard is not very friendly with HAL. Just saving to a CSV file and will plot post-training
log_dir = "logs/" + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)




# model = tf.keras.models.load_model('saved_models/model_epoch_start175_nowAt_03.hdf5')

model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#remember to add initial_epoch back in
model.fit(x_train, y_train, epochs=1500, batch_size=384, validation_data=(x_val, y_val),callbacks=[LossAndErrorPrintingCallback(), tensorboard_callback, checkpoint_callback])



"""##Saving Model"""

#Saving the model into H5 system file

model.save("full_data_set_model_ASL.h5")
print("Model Saved into", "full_data_set_model_ASL.h5")





W0523 22:02:56.061390 140735892507248 callbacks.py:864] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.
W0523 22:02:56.063494 140735892507248 backend.py:548] OMP_NUM_THREADS is no longer used by the default Keras config. To configure the number of threads, use tf.config.threading APIs.


Train on 157 steps, validate on 7 steps
Epoch 1/1500
157/157 [==============================] - 19s 119ms/step - loss: 1.6161 - acc: 0.6179 - val_loss: 3.8215 - val_acc: 0.1224
Epoch 2/1500
157/157 [==============================] - 6s 39ms/step - loss: 1.7091 - acc: 0.5720 - val_loss: 5.0768 - val_acc: 0.1298
Epoch 3/1500
157/157 [==============================] - 6s 39ms/step - loss: 1.5193 - acc: 0.6032 - val_loss: 4.7122 - val_acc: 0.1752
Epoch 4/1500
157/157 [==============================] - 6s 39ms/step - loss: 1.3330 - acc: 0.6558 - val_loss: 8.4883 - val_acc: 0.1276
Epoch 5/1500
157/157 [==============================] - 6s 39ms/step - loss: 1.5526 - acc: 0.5799 - val_loss: 9.3314 - val_acc: 0.1782
Epoch 6/1500
157/157 [==============================] - 6s 39ms/step - loss: 1.3420 - acc: 0.6421 - val_loss: 5.4141 - val_acc: 0.1551
Epoch 7/1500
157/157 [==============================] - 6s 39ms/step - loss: 1.1920 - acc: 0.6839 - val_loss: 8.9545 - val_acc: 0.1068
Epoch 8/1500
